In [ ]:
import sqlite3
import requests
import time
import os
from dotenv import load_dotenv

In [3]:
conn = sqlite3.connect("seq_data.db")
cur = conn.cursor()

In [4]:
cur.execute("""
CREATE TABLE IF NOT EXISTS dim_company (
	cik INTEGER PRIMARY KEY,
	cik_padded TEXT NOT NULL,
	ticker TEXT,
	company_name TEXT NOT NULL
);
""")

In [17]:
load_dotenv()
user_agent = os.getenv('user_agent')

In [18]:
headers = {"User-Agent": user_agent}
url = "https://www.sec.gov/files/company_tickers.json"
res = requests.get(url, headers=headers)
companies = res.json()

In [19]:
company_data = []
for _, c in companies.items():
	company_data.append((
		c['cik_str'],
		str(c['cik_str']).zfill(10),
		c.get('ticker'),
		c['title']
	))

cur.executemany("""
INSERT OR REPLACE INTO dim_company (cik, cik_padded, ticker, company_name)
VALUES (?, ?, ?, ?);
""", company_data)

conn.commit()